In [4]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI

In [5]:
INGREDIENT_SAFETY_DB = {
    "salicylic acid": {
        "safe": True,
        "notes": "Safe for topical use in rinse-off products when used under recommended limits.",
        "max_limit_percent": 2.0
    },
    "niacinamide": {
        "safe": True,
        "notes": "Well tolerated, suitable for daily skincare.",
        "max_limit_percent": 5.0
    },
    "hydroquinone": {
        "safe": False,
        "notes": "Restricted in many countries due to safety concerns.",
        "max_limit_percent": 0.0
    }
}

RECOMMENDED_PERCENTAGE_DB = {
    ("salicylic acid", "face wash"): (0.5, 2.0),
    ("salicylic acid", "serum"): (0.2, 1.0),
    ("niacinamide", "moisturizer"): (2.0, 5.0),
}


In [6]:
from langchain_core.tools import tool

@tool
def check_ingredient_safety(ingredient_name: str) -> dict:
    """
    Check whether an ingredient is safe for cosmetic use.
    Returns structured safety data.
    """
    ingredient = ingredient_name.lower()

    return INGREDIENT_SAFETY_DB.get(
        ingredient,
        {"safe": False, "notes": "No verified safety data available.", "max_limit_percent": None}
    )


@tool
def recommended_percentage(ingredient_name: str, product_type: str) -> dict:
    """
    Returns recommended safe concentration range for an ingredient by product type.
    """

    key = (ingredient_name.lower(), product_type.lower())

    if key in RECOMMENDED_PERCENTAGE_DB:
        low, high = RECOMMENDED_PERCENTAGE_DB[key]
        return {
            "min_percent": low,
            "max_percent": high,
            "notes": "Within dermatologically safe cosmetic limits."
        }
    
    return {
        "min_percent": None,
        "max_percent": None,
        "notes": "No verified recommendation available."
    }

In [ ]:
from langchain_openai import AzureChatOpenAI

load_dotenv()

model = AzureChatOpenAI(
    azure_deployment=os.getenv("DEPLOYMENT_NAME"),
    azure_endpoint=os.getenv("OPENAI_API_ENDPOINT"),
    api_key=os.getenv("OPENAI_API_KEY"),
    api_version=os.getenv("OPENAI_API_VERSION"),
    temperature=0,
    max_retries=5,
)

In [12]:
from langchain_classic.agents import create_tool_calling_agent
from langchain_classic.agents import AgentExecutor
from langchain_core.prompts import ChatPromptTemplate

tools = [check_ingredient_safety, recommended_percentage]

prompt = ChatPromptTemplate.from_messages([
    ("system", 
     "You are a cosmetic formulation assistant. "
     "Use tools to verify safety and ingredient percentages. "
     "Do NOT guess numeric limits. Always call tools when needed."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

agent = create_tool_calling_agent(model, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True
)


In [13]:
query = "Can I use salicylic acid in a daily face wash? At what percentage?"

response = agent_executor.invoke({"input": query})

print(response["output"])




> Entering new AgentExecutor chain...

Invoking: `check_ingredient_safety` with `{'ingredient_name': 'salicylic acid'}`


{'safe': True, 'notes': 'Safe for topical use in rinse-off products when used under recommended limits.', 'max_limit_percent': 2.0}
Invoking: `recommended_percentage` with `{'ingredient_name': 'salicylic acid', 'product_type': 'face wash'}`


{'min_percent': 0.5, 'max_percent': 2.0, 'notes': 'Within dermatologically safe cosmetic limits.'}Salicylic acid is safe for use in a daily face wash, provided it is used within recommended limits. The safe concentration range for salicylic acid in a face wash is between **0.5% and 2.0%**. It is important to stay within this range to ensure safety and effectiveness.

> Finished chain.
Salicylic acid is safe for use in a daily face wash, provided it is used within recommended limits. The safe concentration range for salicylic acid in a face wash is between **0.5% and 2.0%**. It is important to stay within this range to ensure 